# Training COSDD<br>
In this notebook, we will train a model to remove row correlated and signal-dependent imaging noise. 

In [1]:
import os
import urllib
from zipfile import ZipFile 

import torch
import tifffile
import lightning as L
from lightning.pytorch.callbacks import EarlyStopping
from lightning.pytorch.loggers import TensorBoardLogger
import matplotlib.pyplot as plt
import numpy as np

import utils
from lvae.models.lvae import LadderVAE
from ar_decoder.pixelcnn import PixelCNN
from s_decoder import SDecoder
from direct_denoiser.models.unet import UNet
from dvlae import DVLAE

%matplotlib inline

In [2]:
use_cuda = torch.cuda.is_available()

## 1. Download and load data

In this example we will be using the Mito-Confocal dataset, provided by: <br>
Hagen, G.M., Bendesky, J., Machado, R., Nguyen, T.A., Kumar, T. and Ventura, J., 2021. Fluorescence microscopy datasets for training deep neural networks. GigaScience, 10(5), p.giab032.

In [3]:
# create a folder for our data.
if not os.path.exists("./data"):
    os.mkdir("./data")

# Check if the file has already been downloaded
if not os.path.exists(os.path.join("./data/Fluo-C3DH-A549.zip")):
    # Download the file
    url = "http://data.celltrackingchallenge.net/training-datasets/Fluo-C3DH-A549.zip"
    urllib.request.urlretrieve(url, "./data/Fluo-C3DH-A549.zip")

    # Unzip the file
    with ZipFile("./data/Fluo-C3DH-A549.zip", "r") as zip_ref:
        zip_ref.extractall("./data")

# Go into the folder and stack all of the tiffs into one pytorch tensor
data_dir = "./data/Fluo-C3DH-A549/01"
files = os.listdir(data_dir)
images = [f for f in files if f.endswith(".tif")]
images.sort()
low_snr = [tifffile.imread(os.path.join(data_dir, f)) for f in images]
low_snr = np.stack(low_snr).astype(float)[:, np.newaxis]  # Add channel dimension
low_snr = torch.tensor(low_snr, dtype=torch.float32)

print(f"Noisy data size: {low_snr.size()}")

Noisy data size: torch.Size([30, 1, 29, 300, 350])


To use your own data, load it as a `torch.Tensor` with dimensions: [Number of images, Channels, Depth, Height, Width], then assign it to the `low_snr` variable. Make sure the data type is float32.<br>

## 2. Examine spatial correlation of the noise

To apply this denoiser, the noise must be either spatially uncorrelated or correlated along the x, y or z axis. The sample autocorrelation plot can be used to determine whether this is the case.

First, we'll look at a slice of the noisy data.

In [4]:
vmin = np.percentile(low_snr.numpy(), q=1)
vmax = np.percentile(low_snr.numpy(), q=99)

In [5]:
plt.figure(figsize=(10, 10))
plt.imshow(low_snr[0, 0, 10, :, :], cmap="gray", vmin=vmin, vmax=vmax)
plt.show()

Samples of pure noise are needed to estimate its spatial autocorrelation. Dark image patches with no signal are best for this. For this example dataset, we've collected a list of such patches. The more noise pixels used, the more accurate the estimate.

In [6]:
dark_patches = [low_snr[:, 0, 0, :, :]]

In [7]:
noise_ac = utils.autocorrelation(dark_patches, max_lag=25)

In [8]:
plt.figure(figsize=(10, 10))
plt.imshow(noise_ac, cmap="seismic", vmin=-1, vmax=1)
plt.colorbar()
plt.title("Autocorrelation of the noise")
plt.xlabel("Horizontal lag")
plt.ylabel("Vertical lag")
plt.show()

The plot reveals a pattern of spatial correlation along the x axis. To remove this, the autoregressive decoder of our VAE must have a receptive field spanning pixels in the same row. It is not necessary for it to span *all* correlated pixels, just enough to learn an accurate model. We found 40 pixels to work in all of our experiments, which is achieved by giving the decoder a kernel size of 5 and 4 layers.

## 3. Create training and validation dataloaders

The data will be fed to the model by two dataloaders, `train_loader` and `val_loader`, for the training and validation set respectively.
We use virtual batches to reduce memory, were a batch of, e.g., 16 images is split into 16 batches of 1 image and gradients are accumulated over each batch before updating model parameters.

Data augmentation should be passed as an argument to `utils.TrainDataset`. Be careful with the choice of data augmentation. Operations such as rotating and mirroring will change the orientation of noise structures. Only use operations that won't affect the autocorrelation plot above. In this example, we just use random cropping.

`real_batch_size` (int) Number of images passed through the network at a time. <br>
`n_grad_batches` (int) Number of batches to pass through the network before updating the model parameters. <br>
`crop_size` (tuple(int)) The (depth, height and width) of randomly cropped patches. Should be less than the dimensions of your images. It is recommended that these sizes are powers of 2.<br>
`train_split` (0\<float\<1) Fraction of images to be used in the training set, with the remainder used for the validation set.


In [10]:
real_batch_size = 1
n_grad_batches = 16
print(f"Effective_batch_size: {real_batch_size * n_grad_batches}")
crop_size = (16, 128, 128)
train_split = 0.9

n_iters = (low_snr.shape[-3] * low_snr.shape[-2] * low_snr.shape[-1]) // (crop_size[0] * crop_size[1] * crop_size[2])
transform = utils.RandomCrop3D(crop_size)

low_snr = low_snr[torch.randperm(len(low_snr))]
train_set = low_snr[: int(len(low_snr) * train_split)]
val_set = low_snr[int(len(low_snr) * train_split) :]

train_set = utils.TrainDataset(train_set, n_iters=n_iters, transform=transform)
val_set = utils.TrainDataset(val_set, n_iters=n_iters, transform=transform)

train_loader = torch.utils.data.DataLoader(
    train_set, batch_size=real_batch_size, shuffle=True, pin_memory=True
)
val_loader = torch.utils.data.DataLoader(
    val_set, batch_size=real_batch_size, shuffle=False, pin_memory=True
)

Effective_batch_size: 16


## 4. Create the models

<div class="alert alert-block alert-info">

####  💡 Memory consideration
The Ladder VAE and Direct Denoiser both optionally downsample inputs at each level by halving their spatial dimensions. Before each pass, they will check whether the input is large enough for the prescribed number of downsamples. If it is not, each spatial dimensions will be padded up to the nearest power of 2 that meets this requirement. This makes applying the network to arbitrary image sizes easier, but the padding will take up valuable space in memory. 

To avoid wasting VRAM, ensure image patches fed to the network during training are powers of 2 (`crop_size` in the cell above) and set the number of downsampling operations to a value small enough for this size (`downsampling` in the cell below). To give more fine grained control, downsampling operations can be set per dimension (depth, height, width). For example, to instruct the network to half the height and width of the image, but not the depth, at every level of a three level network, set 
```
downsampling = [[False, True, True], [False, True, True], [False, True, True]]
```
</div>



`lvae` The ladder variational autoencoder that will output latent variables.<br>
* `s_code_channels` (int) Number of channels in outputted latent variable.
* `n_layers` (int) Number of levels in the hierarchical vae.
* `z_dims` (list(int)) List with the numer of latent space dimensions at each level of the hierarchy. List starts from the input/output level and works down.
* `downsampling` (list(list(bool))) Whether to halve the spatial size (depth, height, width) of the input tensor at each level of the vae. This is controlled per dimension, so to halve each dimension once at each level of the vae 

`ar_decoder` The autoregressive decoder that will decode latent variables into a distribution over the input.<br>
* `kernel_size` (int) Length of 1D convolutional kernels.
* `noise_direction` (str) This needs to match the orientation of the noise structures we revealed in the autocorrelation plot in Part 2. If the noise is correlated along either the x, y or z axis, set this to "x", "y" or "z" respectively.
* `n_filters` (int) Number of feature channels.
* `n_gaussians` (int) Number of components in Gaussian mixture used to model data.

`s_decoder` A decoder that will map the latnet variables into image space. <br>
* `n_filters` (int) The number of feature channels.<br>

`direct_denoiser` The U-Net that can optionally be trained to predict the MMSE or MMAE of the denoised images. This will slow training slightly but massively speed up inference and is worthwile if you have an inference dataset in the gigabytes. See [this paper](https://arxiv.org/abs/2310.18116). The network architecture is a deterministic version of the LadderVAE. Enable or disable the direct denoiser by setting `use_direct_denoiser` to `True` or `False`.
* `n_filters` (int) Feature channels at each level of UNet. Doubles at subsequent layers.
* `n_layers` (int) Depth of the U-Net.
* `downsampling` (list(int)) Binary list of whether to downsample at each level of the hierarchy. 1 for do and 0 for don't.
* `loss_fn` (str) Whether to use `"L1"` or `"L2"` loss function to predict either the mean or pixel-wise median of denoised images respectively.

`dvlae` The container that will unify and train the above models.
* `n_grad_batches` (int) Number of batches to accumulate gradients for before updating weights of all models. If the batch or random crop size has been reduced to lower memory consumption, increase this value for the effective batch size to stay the same.

Note that some hyperparameters have been set to low values for reduced memory consumption. To use the full size model, use the value suggested in the inline comment.

In [11]:
s_code_channels = 32  # Set this to 128 to use the full size model

n_layers = 6  # Set this to 14 to use the full size model
z_dims = [s_code_channels // 2] * n_layers
downsampling = [[False, True, True]] * 2 + [[True, True, True]] * (n_layers - 2) 
lvae = LadderVAE(
    colour_channels=low_snr.shape[1],
    img_shape=crop_size,
    s_code_channels=s_code_channels,
    n_filters=s_code_channels,
    z_dims=z_dims,
    downsampling=downsampling,
)

ar_decoder = PixelCNN(
    colour_channels=low_snr.shape[1],
    s_code_channels=s_code_channels,
    kernel_size=5,
    noise_direction="x",
    n_filters=64,
    n_layers=4,
    n_gaussians=5,
)

s_decoder = SDecoder(
    colour_channels=low_snr.shape[1],
    s_code_channels=s_code_channels,
    n_filters=s_code_channels,
)

use_direct_denoiser = True
if use_direct_denoiser:
    direct_denoiser = UNet(
        colour_channels=low_snr.shape[1],
        n_filters=s_code_channels,
        n_layers=n_layers,
        downsampling=downsampling,
        loss_fn="L2",
    )
else:
    direct_denoiser = None

dvlae = DVLAE(
    vae=lvae,
    ar_decoder=ar_decoder,
    s_decoder=s_decoder,
    direct_denoiser=direct_denoiser,
    data_mean=low_snr.mean(),
    data_std=low_snr.std(),
    n_grad_batches=n_grad_batches,
)

/home/ben/miniforge3/envs/autonoise/lib/python3.10/site-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'vae' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['vae'])`.
/home/ben/miniforge3/envs/autonoise/lib/python3.10/site-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'ar_decoder' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['ar_decoder'])`.
/home/ben/miniforge3/envs/autonoise/lib/python3.10/site-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 's_decoder' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['s_decoder'])`.
/home/ben/miniforge3/envs/autonoise/lib/python3.10/site-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'direct_d

## 5. Train the model

`model_name` should be set to something appropriate so that the trained parameters can be used later for inference.<br>
`max_epochs` The number of training epochs.<br>
`patience` If the validation loss has plateaued for this many epochs, training will stop.

Training logs can be monitored on Tensorboard. Run the three cells below to activate it in the notebook. Alternatively, open a terminal, activate an environment with Tensorboard installed and enter `tensorboard --logdir path/to/COSDD/checkpoints` then open a browser and enter localhost:6006.

In the SCALARS tab, there will be 4 metrics to track (5 if direct denoiser is enabled). These are:<br>
1. `kl_loss` The Kullback-Leibler divergence between the VAE's approximate posterior and its prior. This can be thought of as a measure of how much information about the input image is going into the VAE's latent variables. We want information about the input's underlying clean signal to go into the latent variables, so this metric shouldn't go all the way to zero. Instead, it can typically go either up or down during training before plateauing.<br>
2. `reconstruction_loss` The negative log-likelihood of the AR decoder's predicted distribution given the input data. This is how accurately the AR decoder is able to predict the input. This value can go below zero and should decrease throughout training before plateauing.<br>
3. `elbo` The Evidence Lower Bound, which is the total loss of the main VAE. This is the sum of the kl reconstruction loss and should decrease throughout training before plateauing.<br>
4. `sd_loss` The mean squared error between the noisy image and the image predicted by the signal decoder. This metric should steadily decrease towards zero without ever reaching it. Sometimes the loss will not go down for the first few epochs because its input (produced by the VAE) is rapidly changing. This is ok and the loss should start to decrease when the VAE stabilises. <br>
5. `dd_loss` The mean squared error between the output of the direct denoiser and the clean images predicted by the signal decoder. This will only be present if `use_direct_denoiser` is set to `True`. The metric should steadily decrease towards zero without ever reaching it, but may be unstable at the start of training as its targets (produced by the signal decoder) are rapidly changing.

There will also be an IMAGES tab. This shows noisy input images from the validation set and some outputs. These will be two randomly sampled denoised images (sample 1 and sample 2), the average of ten denoised images (mmse) and if the direct denoiser is enabled, its output (direct estimate).

If noise has not been fully removed from the output images, try increasing `n_gaussians` argument of the AR decoder. This will give it more flexibility to model complex noise characteristics. However, setting the value too high can lead to unstable training. Typically, values from 3 to 5 work best.

Note that the trainer is set to train for only 20 minutes in this example. Remove the line with `max_time` to train fully.

In [12]:
model_name = "Fluo-C3DH-A549"
checkpoint_path = os.path.join("checkpoints", model_name)
tb_logger = TensorBoardLogger(checkpoint_path)

max_epochs = 1000
patience = 100

trainer = L.Trainer(
    logger=tb_logger,
    accelerator="gpu" if use_cuda else "cpu",
    devices=1,
    max_epochs=max_epochs,
    log_every_n_steps=len(train_set) // real_batch_size,
    callbacks=[EarlyStopping(patience=patience, monitor="val/elbo")],
    precision="bf16-mixed" if use_cuda else "fp32",
)

/home/ben/miniforge3/envs/autonoise/lib/python3.10/site-packages/lightning/fabric/connector.py:563: `precision=bf16` is supported for historical reasons but its usage is discouraged. Please set your precision to bf16-mixed instead!
Using bfloat16 Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [13]:
trainer.fit(dvlae, train_loader, val_loader)
trainer.save_checkpoint(os.path.join(checkpoint_path, "final_model.ckpt"))

You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name            | Type      | Params
----------------------------------------------
0 | vae             | LadderVAE | 2.0 M 
1 | ar_decoder      | PixelCNN  | 195 K 
2 | s_decoder       | SDecoder  | 83.9 K
3 | direct_denoiser | UNet      | 1.5 M 
----------------------------------------------
3.8 M     Trainable params
0         Non-trainable params
3.8 M     Total params
15.284    Total estimated model params size (MB)


Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

/home/ben/miniforge3/envs/autonoise/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=35` in the `DataLoader` to improve performance.


/home/ben/miniforge3/envs/autonoise/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=35` in the `DataLoader` to improve performance.


Epoch 125: 100%|██████████| 297/297 [00:46<00:00,  6.35it/s, v_num=88]
